In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import glob
import numpy as np
import geopandas as gpd
from src.utils import Data, markdown_html
from bokeh.io import show, save, output_file
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider, BasicTicker)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.palettes import Inferno256
from bokeh.plotting import figure

In [6]:
import os
os.environ['SHAPE_RESTORE_SHX']='YES'

In [7]:
shp = gpd.read_file('data/tl_2019_us_zcta510.shp')

CRSError: Invalid projection: epsg:4269: (Internal Proj Error: proj_create: cannot build geodeticCRS 4269: SQLite error on SELECT name, ellipsoid_auth_name, ellipsoid_code, prime_meridian_auth_name, prime_meridian_code, area_of_use_auth_name, area_of_use_code, publication_date, deprecated FROM geodetic_datum WHERE auth_name = ? AND code = ?: no such column: publication_date)

In [7]:
maryland = Data(state='MD')

KeyboardInterrupt: 

In [54]:
data = maryland.geo\
    .merge(maryland.zip_map, on ='Zip', how = 'right')\
    .merge(maryland.zip_covid, on ='Zip', how ='left')\
    .merge(maryland.zip_population.assign(Zip = lambda d: d.Zip.astype(int)), on ='Zip') \
    .assign(Date = lambda d: d.Date.fillna(d.Date.max())) \
    .filter(['Zip','City','State','Cases','Date','geometry', 'Population']) 

total_case_data = data \
    .pipe(lambda d: d[d.Date==d.Date.max()]) \
    .assign(Cases = lambda d: d.Cases.fillna(0)) \
    .pipe(lambda d: d[~pd.isnull(d.geometry)]) \
    .assign(per_population = lambda d: d.Cases / d.Population.astype(int) * 1000)
today = str(total_case_data.Date.astype(str).unique()[0])

per_day_increase_data =  data \
    .groupby(['Zip'], as_index=False)\
    .apply(lambda d: d.nlargest(2, 'Date') \
                    .assign(increase = lambda d: d.Cases.max() - d.Cases.min()) \
                    .assign(increase = lambda d: np.where(pd.isnull(d.increase), d.Cases, d.increase))\
                    .assign(increase = lambda d: d.increase.fillna(0))\
                    .pipe(lambda d: d[d.Date == d.Date.max()])) \
    .pipe(lambda d: d[~pd.isnull(d.geometry)]) \
    .assign(per_population = lambda d: d.increase / d.Population.astype(int) * 1e6) \
    .assign(Date = lambda d: d.Date.astype(str))

In [47]:
geosource = GeoJSONDataSource(geojson = total_case_data.drop('Date', axis=1).to_json())
color_mapper = LinearColorMapper(palette=Inferno256, 
                           low=total_case_data.per_population.min(), 
                           high=total_case_data.per_population.max())

p1 = figure(title = 'COVID19 in Maryland\n(Cases per 1,000 people)', 
           plot_height = 600,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p1.xgrid.grid_line_color = None
p1.ygrid.grid_line_color = None
p1.axis.visible = False
p1.title.text_font_size = '25pt'

# Add patch renderer to figure.
states = p1.patches('xs','ys', source = geosource,
                   fill_color = {'field':'per_population',
                                'transform':color_mapper},
                   line_color = 'gray', 
                   line_width = 0.25, 
                   fill_alpha = 1)
# Create hover tool
p1.add_tools(HoverTool(tooltips = [('City','@City'),
                                  ('Zip code','@Zip'),
                                  ('Population','@Population'),
                                  ('Cases', '@Cases'),
                                  ('Cases/1k population', '@per_population')]))

#color bar
color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 500, height = 20,
                     border_line_color = None,
                     location = (0,0), 
                    orientation='horizontal')
p1.add_layout(color_bar)
show(p1)

In [55]:
geosource = GeoJSONDataSource(geojson = per_day_increase_data.to_json())
color_mapper = LinearColorMapper(palette=Inferno256, 
                           low=per_day_increase_data.increase.min(), 
                           high=per_day_increase_data.increase.max())

p1 = figure(title = 'Daily increase in COVID19 cases in Maryland', 
           plot_height = 600,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p1.xgrid.grid_line_color = None
p1.ygrid.grid_line_color = None
p1.axis.visible = False
p1.title.text_font_size = '25pt'

# Add patch renderer to figure.
states = p1.patches('xs','ys', source = geosource,
                   fill_color = {'field':'increase',
                                'transform':color_mapper},
                   line_color = 'gray', 
                   line_width = 0.25, 
                   fill_alpha = 1)
# Create hover tool
p1.add_tools(HoverTool(tooltips = [('Date','@Date'),
                                   ('City','@City'),
                                  ('Zip code','@Zip'),
                                  ('Population','@Population'),
                                  ('New cases', '@increase'),
                                  ('New cases/1M population', '@per_population')]))

#color bar
color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 500, height = 20,
                     border_line_color = None,
                     location = (0,0), 
                    orientation='horizontal')
p1.add_layout(color_bar)
show(p1)

In [62]:
ts_data = maryland.zip_covid\
    .merge(maryland.zip_map.filter(['Zip','City']), on = 'Zip')\
    .groupby(['Zip','City'], as_index=False)\
    .apply(lambda d: d.sort_values('Date')\
                   .assign(Cases = lambda d: d.Cases.fillna(method='ffill', axis=0)\
                   .fillna(0)))\
    .reset_index(drop=True) \
    .groupby(['City','Date'], as_index=False)\
    .agg({'Cases':'sum'})\
    .assign(formatted_date = lambda d: d.Date.astype(str)) 
ts_data.head()

,City,Date,Cases,formatted_date
0,Aberdeen,2020-04-15,8,2020-04-15
1,Aberdeen,2020-04-16,8,2020-04-16
2,Aberdeen,2020-04-17,8,2020-04-17
3,Abingdon,2020-04-13,14,2020-04-13
4,Abingdon,2020-04-14,14,2020-04-14


In [63]:
p2 = figure(x_axis_type="datetime", 
            plot_width = 950, 
            x_axis_label='Date',
            y_axis_label = 'Total Cases',
            title = 'Daily cases by city',
            tools='save,pan,box_zoom,reset,wheel_zoom')
p2.xgrid.grid_line_color = None
p2.ygrid.grid_line_color = None
p2.title.text_font_size = '25pt'
p2.xaxis.axis_label = 'Date'
p2.yaxis.axis_label = 'Total Cases'
p2.xaxis.axis_label_text_font_size = "25pt"
p2.xaxis.major_label_text_font_size = "25pt"
p2.yaxis.axis_label_text_font_size = "25pt"
p2.yaxis.major_label_text_font_size = "25pt"


for zip, zip_df in ts_data.groupby('City'):
    source = ColumnDataSource(zip_df)
    p2.line(x='Date',
            y='Cases',
            color = 'grey',
            line_width=4,
            line_alpha=0.8,
            source=source)
    #add tool tips
    hover = HoverTool(tooltips =[
                     ('Date','@formatted_date'),
                     ('Cases','@Cases'),
                     ('City', '@City')])
p2.add_tools(hover)

In [64]:
p = column(p1, p2)
output_file('output.html')
save(p)

INFO:bokeh.io.state:Session output file 'output.html' already exists, will be overwritten.


'/Users/wckdouglas/codes/covid/output.html'

In [65]:
markdown_html('output.html','COVID.html')

INFO:Data collector:Written 84 lines from 85 lines to COVID.html
